#Automatically initializing instance variables in class definitions

One often has to write definitions of Python classes that take several initializing arguments for instance variables. Usually, this looks something like:

    class MyClass():
        def __init__(self, a, b, c, ...):
            self.a = a
            self.b = b
            self.c = c
            ...
            
In order to automate the process of initializing instance variables, we can use function decorators (see [this]([use function decorators](http://stackoverflow.com/questions/1389180/python-automatically-initialize-instance-variables) StackOverflow answer for further discussion):

In [2]:
from functools import wraps
import inspect

def autoinit(init_func):
    """
    Automatically assigns the parameters.

    >>> class process:
    ...     @initializer
    ...     def __init__(self, cmd, reachable=False, user='root'):
    ...         pass
    >>> p = process('halt', True)
    >>> p.cmd, p.reachable, p.user
    ('halt', True, 'root')
    """
    names, varargs, keywords, defaults = inspect.getargspec(init_func)

    @wraps(init_func)
    def wrapper(self, *args, **kargs):
        for name, arg in list(zip(names[1:], args)) + list(kargs.items()):
            setattr(self, name, arg)

        if defaults != None:
            for name, default in zip(reversed(names), reversed(defaults)):
                if not hasattr(self, name):
                    setattr(self, name, default)

        init_func(self, *args, **kargs)

    return wrapper